<a href="https://colab.research.google.com/github/chiyeon01/CNN_Model_Mechanism/blob/main/pytorch/deepfake_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchvision

In [2]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 42.6 MB/s eta 0:00:00


In [3]:
!pip install torchinfo

In [4]:
!wget https://raw.githubusercontent.com/chiyeon01/CNN_Model_Mechanism/refs/heads/main/pytorch/utils.py

--2025-12-21 08:27:33--  https://raw.githubusercontent.com/chiyeon01/CNN_Model_Mechanism/refs/heads/main/pytorch/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11259 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.00K  --.-KB/s    in 0s      

2025-12-21 08:27:33 (80.7 MB/s) - ‘utils.py’ saved [11259/11259]



In [5]:
# Module import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torch.optim import SGD, Adagrad, RMSprop, Adam, AdamW
import torchmetrics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from PIL import Image
from utils import Trainer, Predictor, Custom_Dataset, create_pretrained_model

# Test Module
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:", path)

100%|██████████| 1.68G/1.68G [00:14<00:00, 124MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1


In [9]:
# Config & Settings
class Config:
    batch_size = 16
    FineTune = False
    shuffle = False
    first_learning_rate = 1e-3
    second_learning_rate = 1e-4
    image_size = [224, 224] # 이미지는 3채널로 간주(image is 3 channels)
    model_name = 'efficientnet_b0'
    make_summary = False
    callbacks = []
    metric = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)
    classifier_layer = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_features=1280, out_features=2)
        )

In [13]:
# 모델 생성
model = create_pretrained_model(
    model_name = Config.model_name,
    classifier_layer = Config.classifier_layer,
    make_summary = Config.make_summary
)

# albumentation 기반 transform 생성.
transform = A.Compose([
    A.Resize(*Config.image_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=Config.first_learning_rate)

trainer = Trainer(model=model, loss_fn=loss_fn, metric=Config.metric, optimizer=optimizer)

history = trainer.fit(epochs=1, train_dataloader=train_dataloader, val_dataloader=test_dataloader)